<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
import time
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import shutil
from google.colab import files

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%%time
!rm -r /content/cloned-repo
%cd ..
!git clone -l -s git://github.com/kavyajeetbora/EAST.git /content/cloned-repo
%cd /content/cloned-repo
!ls

rm: cannot remove '/content/cloned-repo': No such file or directory
/
Cloning into '/content/cloned-repo'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 176 (delta 52), reused 0 (delta 0), pack-reused 82
Receiving objects: 100% (176/176), 6.97 MiB | 10.12 MiB/s, done.
Resolving deltas: 100% (97/97), done.
/content/cloned-repo
dataset.py		    EAST_resnet.py		    model.py
detect.py		    EAST_training.ipynb		    model_resnet.py
dsiplay_ground_truth.ipynb  EAST_training_vgg16_bn_2.ipynb  train.py
EAST_detection.ipynb	    loss.py
CPU times: user 120 ms, sys: 39.4 ms, total: 159 ms
Wall time: 7.29 s


In [0]:
from dataset import *

In [5]:
%%time

# AI4Bharat dataset
image_folder = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Test Set/Images'
label_folder = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Test Set/Annotations'

training_dataset = custom_dataset(image_folder, label_folder)
print(len(training_dataset))

428
CPU times: user 3.67 ms, sys: 1.51 ms, total: 5.18 ms
Wall time: 14.3 ms


## Loss for one batch

In [6]:
%%time
train_loader = DataLoader(training_dataset, shuffle = True, batch_size=12, drop_last=True)
imgs, gt_score, gt_geo, ignored_maps = next(iter(train_loader))
print(imgs.size(), gt_score.size(), gt_geo.size(), ignored_maps.size())
print(torch.sum(gt_score))

torch.Size([12, 3, 512, 512]) torch.Size([12, 1, 128, 128]) torch.Size([12, 5, 128, 128]) torch.Size([12, 1, 128, 128])
tensor(13729.)
CPU times: user 6.72 s, sys: 6.81 s, total: 13.5 s
Wall time: 11.8 s


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [8]:
%%time
from model import EAST
model = EAST()
pretrained_path = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/EAST scene text detector/east_vgg16.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
model = model.to(device)
imgs, gt_score, gt_geo, ignored_maps = imgs.to(device), gt_score.to(device), gt_geo.to(device), ignored_maps.to(device)
pred_score, pred_geo = model(imgs)

print(pred_score.size(), pred_geo.size())

torch.Size([12, 1, 128, 128]) torch.Size([12, 5, 128, 128])
CPU times: user 6.39 s, sys: 1.23 s, total: 7.61 s
Wall time: 7.57 s


In [9]:
from loss import Loss

loss_fn = Loss()
loss_fn(gt_score, pred_score, gt_geo, pred_geo, ignored_maps)

classify loss is 0.68417346, angle loss is 0.00623027, iou loss is 0.81353444


tensor(1.5600, device='cuda:0', grad_fn=<AddBackward0>)

## Training Full Batch 

fine tuning pretrained model [east_vgg16.pth](https://drive.google.com/file/d/1AFABkJgr5VtxWnmBU3XcfLJvpZkC2TAg/view)

In [0]:
model = EAST()
pretrained_path = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/EAST scene text detector/east_vgg16.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
model = model.to(device)

In [0]:
def train(model, train_img_path, train_gt_path, batch_size, lr, epoch_iter, num_workers=4, interval=1):
    file_num = len(os.listdir(train_img_path))
    print(file_num)
    trainset = custom_dataset(train_img_path, train_gt_path)
    train_loader = data.DataLoader(trainset, batch_size=batch_size, \
                                   shuffle=True, num_workers=num_workers, drop_last=True)
	
    criterion = Loss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=epoch_iter//2, gamma=0.5)
    loss_curve = []
    
    for epoch in range(epoch_iter):
      model.train()
      scheduler.step()
      epoch_loss = 0
      epoch_time = time.time()
      for i, (img, gt_score, gt_geo, ignored_map) in enumerate(train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
                  epoch+1, epoch_iter, i+1, int(file_num/batch_size), time.time()-start_time, loss.item()))
        
      print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/batch_size), time.time()-epoch_time))
      print(time.asctime(time.localtime(time.time())))
      print('='*50)
      
      
      if (epoch + 1) % interval == 0:
        state_dict = model.state_dict()
        torch.save(state_dict, 'model_epoch_{}.pth'.format(epoch+1))
       
      ## plotting the loss curve
      loss_curve.append(epoch_loss/int(file_num/batch_size))

    return loss_curve

In [0]:
%%time
loss_curve = train(model, image_folder, label_folder, batch_size=8, lr=1e-5, epoch_iter=5, interval = 1)

428


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


classify loss is 0.74742675, angle loss is 0.04168005, iou loss is 1.15757704
Epoch is [1/5], mini-batch is [1/53], time consumption is 0.57793260, batch_loss is 2.32180429
classify loss is 0.75206274, angle loss is 0.07170939, iou loss is 1.14091134
Epoch is [1/5], mini-batch is [2/53], time consumption is 0.22680783, batch_loss is 2.61006808
classify loss is 0.82933325, angle loss is 0.02486178, iou loss is 0.98071146
Epoch is [1/5], mini-batch is [3/53], time consumption is 0.28853297, batch_loss is 2.05866241
classify loss is 0.55255938, angle loss is 0.02387484, iou loss is 0.77025127
Epoch is [1/5], mini-batch is [4/53], time consumption is 0.23708653, batch_loss is 1.56155908
classify loss is 0.90626973, angle loss is 0.02995767, iou loss is 1.21066606
Epoch is [1/5], mini-batch is [5/53], time consumption is 0.23147821, batch_loss is 2.41651249
classify loss is 0.73785579, angle loss is 0.00598528, iou loss is 0.80434263
Epoch is [1/5], mini-batch is [6/53], time consumption is

In [0]:
plt.plot(loss_curve)

1. [**Adam and AdamW algorithms**](https://www.fast.ai/2018/07/02/adam-weight-decay/)

2. [**The optimal learning rate during fine-tuning of an artificial neural network**](https://www.mikulskibartosz.name/the-optimal-learning-rate-during-fine-tuning-of-an-artificial-neural-network/)